# Goal: simulate market dataset such that we can correctly estimate coefficients later on 



In [1]:
import import_ipynb
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 50
n_product = 5
n_consumer = 100
market_variance = 0.01 #amount of variation between markets 
endogenous_noise = 0.01 #how noise to generate price 

#structural parameters 
true_alpha = 3 #decrease with utility 
true_beta = 13 #increase with utility
true_sigma_alpha = 0.7 
true_sigma_beta = 0.3
market_variance = 0.01 #amount of variation between markets 

In [2]:
all_markets = []
for market in range(n_market):
    x = np.random.uniform(0, 1, n_product)
    instrument = np.random.uniform(0, 10, n_product)

    # generate market shares and price variables endogenously 
    price = f.price_gen(x, instrument, endogenous_noise)
    inner_utility = f.utility_gen([true_alpha, true_beta], [true_sigma_alpha, true_sigma_beta], x, price)
    inner_market_utility = f.market_gen(inner_utility, market_variance)
    inner_prob_choice = f.choice_prob(pd.DataFrame(inner_market_utility))
    # generate market share directly from the choice probabilities
    outer_market_share = inner_prob_choice.sum(axis=0)/n_consumer

    market_j = pd.DataFrame({
        'market_id': market,
        'product_id': list(range(n_product)),
        'x': x, 
        'instrument': instrument, 
        'price': price,
        'share': outer_market_share, 
        'share_not_buy': 1 - sum(outer_market_share)
    })
    # print(market)
    all_markets.append(market_j)

print(all_markets[:50])

export_dataset = pd.concat(all_markets, ignore_index=True)
export_dataset.to_csv('simulated_data.csv', index=False)

[   market_id  product_id         x  instrument     price     share  \
0          0           0  0.365082    7.193515  4.430910  0.000194   
1          0           1  0.834273    6.964678  4.494451  0.075640   
2          0           2  0.457430    5.667688  3.578917  0.005508   
3          0           3  0.985595    7.382508  4.803169  0.259659   
4          0           4  0.480791    6.961709  4.363687  0.001031   

   share_not_buy  
0       0.657967  
1       0.657967  
2       0.657967  
3       0.657967  
4       0.657967  ,    market_id  product_id         x  instrument     price     share  \
0          1           0  0.873922    8.799675  5.689626  0.001962   
1          1           1  0.587474    2.440697  1.746617  0.418067   
2          1           2  0.499731    1.907650  1.374154  0.412491   
3          1           3  0.351967    1.553419  1.079486  0.153124   
4          1           4  0.149909    1.590129  1.010169  0.013848   

   share_not_buy  
0       0.000508  
1   